## # 20.02.06 통계량 살피기
- Trend correlation / error_mse_ / (error_ : error는 기준이 아니지만 , 일단 같이 구했었다)

## ## 작업 스텝
- 모델링 데이터 미리 load
- 변수들 통계량 살피기
- 선택된 변수들 eda
- 통계량을 통해 선택된 변수들로 모델링

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import glob
import os
import scipy
import pickle

%matplotlib inline

In [2]:
pd.__version__

'1.0.1'

In [3]:
scipy.__version__

'1.4.1'

In [ ]:
import seaborn as sns
import pylift
import seaborn

In [5]:
from sklearn.model_selection import train_test_split
from collections import Counter
import featexp

import multiprocessing
import datetime

In [6]:
pwd

'/mnt/sda1/myeonggyulee/myeonggyulee/uplift/2020_0113/ver5_0203_trendcorr'

In [8]:
pd.set_option('display.max_columns', 500)

---

## ## Trend correlation은 엑슬 테이블을 참고한다



---



---



# 1. 모델링 - 데이터 로드
- 전처리된 데이터 : preprocessed_uplift_191223.csv


In [9]:
data = pd.read_csv('../preprocessed_uplift_191223.csv', index_col = 0)

In [10]:
data.head()

,crm_id,cmpn_nm,m_id,cont_group,start_date,end_date,min_value,max_value,text_yn,push_yn,mail_yn,last_buy_date,result_buy_yn,buy_period,buy_cnt_date,buy_cnt_payment,buy_amount,buying_trend,buying_trend_group,cate_buy_cnt,cate_rate,coupon_use_rate,buy_amount_3day,buy_cnt_payment_3day,buy_ep_yn_3day,buy_amount_avg,visit_gubn,dt_cnt_last_visit,dt_cnt_last_buy,last_buy_seg,last_buy_platform,wmp_pay_yn,ratio_spc,referral_source,referral_flag,refund_rate,wmp_freq,wmp_stay_avg,order_route,wmp_pick_click,rgt_elapsed_days,buy_amount_avg_all,buy_amount_avg_all_seg,rgt_elapsed_days_seg,cate_main_buy_,text_mail_wemakepay,text_mail_wemakepay_bool,birth,click_cnt,search_cnt,download_coupon_cnt,cart_cnt,cart_category_cnt,cart_buy_cnt,uv_cnt,ad_uv_yn
0,33,~3day,1000298,0,2019-11-20,2019-11-20,0,0,1,0,1,2019-11-18,1,6.0,40,58,1122105,0.00000,stable,11.0,35.0,41,69000,3,N,19147.0,O,5.0,6.0,D-2,APP,0,0.9,google,outside,11,16.0,7.0,best,5,3330,28052.625000,1,5,3,2,0,30,169,84,11,60,11,30,55,1
1,33,~3day,1000347,1,2019-11-14,2019-11-14,30000,5000,1,0,1,2019-11-13,0,178.0,2,2,37400,0.36854,stable,1.0,50.0,50,27500,1,Y,9900.0,R,177.0,178.0,D-1,MWEB,0,0.0,naver,outside,0,5.0,2.0,else,0,3324,18700.000000,1,5,11,2,0,30,6,0,0,1,1,2,4,1
2,33,~3day,1000571,0,2019-11-14,2019-11-14,0,0,1,1,1,2019-11-13,0,36.0,11,15,240412,-2.00000,long,1.0,20.0,20,11800,1,N,16329.0,O,6.0,36.0,D-1,APP,0,0.2,app_push,inside,0,3.0,3.0,else,0,3324,21855.636364,1,5,11,2,0,40,38,22,2,1,1,4,3,1
3,33,~3day,1001894,1,2019-11-11,2019-11-11,30000,5000,1,0,1,2019-11-08,0,70.0,6,8,90100,24.00000,short,3.0,60.0,38,5000,1,N,12157.0,O,7.0,7.0,D-3,APP,0,1.0,(direct),direct,33,6.0,0.0,else,1,3321,15016.666667,1,5,2,2,0,30,45,25,1,6,6,5,10,1
4,33,~3day,1002503,1,2019-11-12,2019-11-12,30000,5000,1,0,1,2019-11-11,0,7.0,49,69,1090116,0.00000,stable,18.0,40.0,17,18700,2,N,15991.0,O,2.0,31.0,D-1,PC,0,0.8,TypeB_C,outside,7,31.0,3.0,special,0,3322,22247.265306,1,5,3,2,0,80,122,50,16,112,11,42,56,1


In [11]:
data.columns

Index(['crm_id', 'cmpn_nm', 'm_id', 'cont_group', 'start_date', 'end_date',
       'min_value', 'max_value', 'text_yn', 'push_yn', 'mail_yn',
       'last_buy_date', 'result_buy_yn', 'buy_period', 'buy_cnt_date',
       'buy_cnt_payment', 'buy_amount', 'buying_trend', 'buying_trend_group',
       'cate_buy_cnt', 'cate_rate', 'coupon_use_rate', 'buy_amount_3day',
       'buy_cnt_payment_3day', 'buy_ep_yn_3day', 'buy_amount_avg',
       'visit_gubn', 'dt_cnt_last_visit', 'dt_cnt_last_buy', 'last_buy_seg',
       'last_buy_platform', 'wmp_pay_yn', 'ratio_spc', 'referral_source',
       'referral_flag', 'refund_rate', 'wmp_freq', 'wmp_stay_avg',
       'order_route', 'wmp_pick_click', 'rgt_elapsed_days',
       'buy_amount_avg_all', 'buy_amount_avg_all_seg', 'rgt_elapsed_days_seg',
       'cate_main_buy_', 'text_mail_wemakepay', 'text_mail_wemakepay_bool',
       'birth', 'click_cnt', 'search_cnt', 'download_coupon_cnt', 'cart_cnt',
       'cart_category_cnt', 'cart_buy_cnt', 'uv_cnt', 'ad

In [12]:
# 모델링에 사용될 Features 따로 정의한다
# cont_group(잠재변수) / result_buy_yn(종속변수)

used_features = ['cont_group', 'result_buy_yn', 'text_yn', 'push_yn', 'mail_yn',
       'buy_period', 'buy_cnt_date', 'buy_cnt_payment', 
       'buy_amount', 'buying_trend', 'buying_trend_group',
       'cate_buy_cnt', 'cate_rate', 'coupon_use_rate', 'buy_amount_3day',
       'buy_cnt_payment_3day', 'buy_ep_yn_3day', 'buy_amount_avg',
       'visit_gubn', 'dt_cnt_last_visit', 'dt_cnt_last_buy', 'last_buy_seg',
       'last_buy_platform', 'wmp_pay_yn', 'ratio_spc', 
       'referral_flag', 'refund_rate', 'wmp_freq', 'wmp_stay_avg',
       'order_route', 'wmp_pick_click', 'rgt_elapsed_days',
       'buy_amount_avg_all', 'buy_amount_avg_all_seg', 'rgt_elapsed_days_seg',
       'cate_main_buy_', 'text_mail_wemakepay', 'text_mail_wemakepay_bool',
       'birth', 'click_cnt', 'search_cnt', 'download_coupon_cnt', 'cart_cnt',
       'cart_category_cnt', 'cart_buy_cnt', 'uv_cnt', 'ad_uv_yn']

In [13]:
len(set(used_features))

47

In [14]:
# 맞는지 확인 - 맞다
set(data.columns) - set(used_features)

{'cmpn_nm',
 'crm_id',
 'end_date',
 'last_buy_date',
 'm_id',
 'max_value',
 'min_value',
 'referral_source',
 'start_date'}

In [15]:
cate_features = ['text_yn', 'push_yn', 'mail_yn',
       'buying_trend_group', 'buy_ep_yn_3day', 'visit_gubn', 'last_buy_seg',
       'last_buy_platform', 'wmp_pay_yn', 'referral_flag', 
       'order_route', 'buy_amount_avg_all_seg', 'rgt_elapsed_days_seg',
       'cate_main_buy_', 'text_mail_wemakepay_bool', 'birth', 'ad_uv_yn']

In [16]:
len(cate_features)

17

In [17]:
data[cate_features].astype('category').info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 419554 entries, 0 to 419553
Data columns (total 17 columns):
 #   Column                    Non-Null Count   Dtype   
---  ------                    --------------   -----   
 0   text_yn                   419554 non-null  category
 1   push_yn                   419554 non-null  category
 2   mail_yn                   419554 non-null  category
 3   buying_trend_group        419554 non-null  category
 4   buy_ep_yn_3day            419554 non-null  category
 5   visit_gubn                419554 non-null  category
 6   last_buy_seg              419554 non-null  category
 7   last_buy_platform         419554 non-null  category
 8   wmp_pay_yn                419554 non-null  category
 9   referral_flag             419554 non-null  category
 10  order_route               419554 non-null  category
 11  buy_amount_avg_all_seg    419554 non-null  category
 12  rgt_elapsed_days_seg      419554 non-null  category
 13  cate_main_buy_            419

---

In [18]:
numerical_features = list(set(used_features) - set(cate_features) - set(['cont_group', 'result_buy_yn']))

In [19]:
for numerical in numerical_features: # 순서 뒤죽박죽이다 -> 아래와 같이 순서정리
    print(numerical)

refund_rate
uv_cnt
buying_trend
buy_amount_avg
text_mail_wemakepay
buy_amount
cate_rate
wmp_freq
click_cnt
download_coupon_cnt
cart_category_cnt
buy_cnt_payment_3day
cart_cnt
dt_cnt_last_buy
cart_buy_cnt
buy_period
cate_buy_cnt
dt_cnt_last_visit
rgt_elapsed_days
buy_amount_3day
coupon_use_rate
wmp_stay_avg
buy_cnt_date
buy_amount_avg_all
ratio_spc
wmp_pick_click
search_cnt
buy_cnt_payment


In [20]:
# 순서 정리
numerical_features = [
    
    'buy_period',
    'buy_cnt_date',
    'buy_cnt_payment',
    'buy_amount',
    'buying_trend',
    'cate_buy_cnt',
    'cate_rate',
    'coupon_use_rate',
    'buy_amount_3day',
    'buy_cnt_payment_3day',
    'buy_amount_avg',
    'dt_cnt_last_visit',
    'dt_cnt_last_buy',
    'ratio_spc',
    'refund_rate',
    'wmp_freq',
    'wmp_stay_avg',
    'wmp_pick_click',
    'rgt_elapsed_days',
    'buy_amount_avg_all',
    'text_mail_wemakepay',
    
    'click_cnt',
    'search_cnt',
    'download_coupon_cnt',
    'cart_cnt',
    'cart_buy_cnt',
    'cart_category_cnt',
    'uv_cnt'
]

In [21]:
len(numerical_features)

28

---

In [22]:
tmp = data[['start_date', 'result_buy_yn', 'cont_group'] + numerical_features].copy()

In [23]:
tmp.shape

(419554, 31)

---
---

## > 모델링 변수 생성

In [24]:
numerical_features

['buy_period',
 'buy_cnt_date',
 'buy_cnt_payment',
 'buy_amount',
 'buying_trend',
 'cate_buy_cnt',
 'cate_rate',
 'coupon_use_rate',
 'buy_amount_3day',
 'buy_cnt_payment_3day',
 'buy_amount_avg',
 'dt_cnt_last_visit',
 'dt_cnt_last_buy',
 'ratio_spc',
 'refund_rate',
 'wmp_freq',
 'wmp_stay_avg',
 'wmp_pick_click',
 'rgt_elapsed_days',
 'buy_amount_avg_all',
 'text_mail_wemakepay',
 'click_cnt',
 'search_cnt',
 'download_coupon_cnt',
 'cart_cnt',
 'cart_buy_cnt',
 'cart_category_cnt',
 'uv_cnt']

---

### >> (*주의점 ) : buying_trend 는 음수값이 존재한다 -> 0이 최소값이 아니다

In [24]:
# buying_trend 는 음수값이 존재한다 -> 0이 최소값이 아니다
data[data.buying_trend<0].buying_trend.max()

-0.11477000000000001

In [25]:
data[data.buying_trend>0].buying_trend.min()

0.013207

In [26]:
data_tmp = tmp.copy()

data_tmp['buying_trend'].replace(0, 0.001, inplace=True)

data_tmp['cate_buy_cnt'].replace(0, -1, inplace=True)
data_tmp['cate_rate'].replace(0, -1, inplace=True)
data_tmp['coupon_use_rate'].replace(0, -1, inplace=True)
data_tmp['buy_amount_avg'].replace(0, -1, inplace=True)
data_tmp['ratio_spc'].replace(0, -1, inplace=True)
data_tmp['refund_rate'].replace(0, -1, inplace=True)
data_tmp['wmp_stay_avg'].replace(0, -1, inplace=True)
data_tmp['wmp_pick_click'].replace(0, -1, inplace=True)

data_tmp['text_mail_wemakepay'].replace(0, -1, inplace=True)
data_tmp['click_cnt'].replace(0, -1, inplace=True)
data_tmp['search_cnt'].replace(0, -1, inplace=True)
data_tmp['download_coupon_cnt'].replace(0, -1, inplace=True)
data_tmp['cart_cnt'].replace(0, -1, inplace=True)
data_tmp['cart_buy_cnt'].replace(0, -1, inplace=True)
data_tmp['cart_category_cnt'].replace(0, -1, inplace=True)
data_tmp['uv_cnt'].replace(0, -1, inplace=True)

---

###  Permutations : (2)

In [28]:
import itertools

###  1차 : add / subs / division 데이터 생성

In [29]:
permutations_list = [list(x) for x in itertools.permutations( numerical_features , 2)]
combinations_list = [list(x) for x in itertools.combinations( numerical_features , 2)]
print('permutations_length:', len(permutations_list), ', combinations_length:', len(combinations_list))

permutations_length: 756 , combinations_length: 378


In [30]:
permutations_list[0]

['buy_period', 'buy_cnt_date']

In [31]:
data_tmp[permutations_list[0][0]]

0           6.000000
1         178.000000
2          36.000000
3          70.000000
4           7.000000
             ...    
419549      9.089414
419550      9.089414
419551      9.089414
419552      9.089414
419553      9.089414
Name: buy_period, Length: 419554, dtype: float64

In [32]:
%%time
### - 데이터 따로따로 만들어 사용하는 경우
data_permut2_add =   data_tmp[['start_date', 'result_buy_yn', 'cont_group']].copy()
#data_permut2_multi = data_tmp[['start_date', 'result_buy_yn', 'cont_group']].copy()

data_permut2_subs =  data_tmp[['start_date', 'result_buy_yn', 'cont_group']].copy()
data_permut2_divis = data_tmp[['start_date', 'result_buy_yn', 'cont_group']].copy()


############ 변수 합성 ############ - 곱하기는 sklearn.preprocessing.PolynomialFeatures 이용하여 생성하자
for func in ['+', '-', '/']:

    if func == '+':
        for comb in combinations_list:
            col_name = comb[0] +' '+ func + ' ' + comb[1]
            data_permut2_add[col_name] = data_tmp[comb[0]] + data_tmp[comb[1]]
            
    #elif func == '*':
    #    for comb in combinations_list:
    #        col_name = comb[0] +' '+ func + ' ' + comb[1]
    #        data_permut2_multi[col_name] = data_tmp[comb[0]] * data_tmp[comb[1]]
            
    elif func == '-':
        for permut in permutations_list:
            col_name = permut[0] +' '+ func + ' ' + permut[1]
            data_permut2_subs[col_name] = data_tmp[permut[0]] - data_tmp[permut[1]]
            
    elif func == '/':
        for permut in permutations_list:
            col_name = permut[0] +' '+ func + ' ' + permut[1]
            data_permut2_divis[col_name] = data_tmp[permut[0]] / data_tmp[permut[1]]

CPU times: user 9.16 s, sys: 12 s, total: 21.1 s
Wall time: 21.1 s


###  2차 : polynomial 데이터 생성
- 3차(3차 만들면 2차, 1차 데이터도 포함)

In [33]:
from sklearn.preprocessing import PolynomialFeatures

In [34]:
# 기존 방법
poly3 = PolynomialFeatures(degree = 3, include_bias=False)
poly3_data = poly3.fit_transform( data_tmp.drop(columns = ['start_date', 'result_buy_yn', 'cont_group']).values )

poly3_data_df = pd.DataFrame(data = poly3_data , columns = poly3.get_feature_names( data_tmp.drop(columns = ['start_date', 'result_buy_yn', 'cont_group']).columns) )
data_permut2_poly3 = pd.concat([data_tmp[['start_date', 'result_buy_yn', 'cont_group']], poly3_data_df] , axis=1)

In [35]:
data_permut2_poly3.head()

,start_date,result_buy_yn,cont_group,buy_period,buy_cnt_date,buy_cnt_payment,buy_amount,buying_trend,cate_buy_cnt,cate_rate,coupon_use_rate,buy_amount_3day,buy_cnt_payment_3day,buy_amount_avg,dt_cnt_last_visit,dt_cnt_last_buy,ratio_spc,refund_rate,wmp_freq,wmp_stay_avg,wmp_pick_click,rgt_elapsed_days,buy_amount_avg_all,text_mail_wemakepay,click_cnt,search_cnt,download_coupon_cnt,cart_cnt,cart_buy_cnt,cart_category_cnt,uv_cnt,buy_period^2,buy_period buy_cnt_date,buy_period buy_cnt_payment,buy_period buy_amount,buy_period buying_trend,buy_period cate_buy_cnt,buy_period cate_rate,buy_period coupon_use_rate,buy_period buy_amount_3day,buy_period buy_cnt_payment_3day,buy_period buy_amount_avg,buy_period dt_cnt_last_visit,buy_period dt_cnt_last_buy,buy_period ratio_spc,buy_period refund_rate,buy_period wmp_freq,buy_period wmp_stay_avg,buy_period wmp_pick_click,buy_period rgt_elapsed_days,buy_period buy_amount_avg_all,buy_period text_mail_wemakepay,buy_period click_cnt,buy_period search_cnt,buy_period download_coupon_cnt,buy_period cart_cnt,buy_period cart_buy_cnt,buy_period cart_category_cnt,buy_period uv_cnt,buy_cnt_date^2,buy_cnt_date buy_cnt_payment,buy_cnt_date buy_amount,buy_cnt_date buying_trend,buy_cnt_date cate_buy_cnt,buy_cnt_date cate_rate,buy_cnt_date coupon_use_rate,buy_cnt_date buy_amount_3day,buy_cnt_date buy_cnt_payment_3day,buy_cnt_date buy_amount_avg,buy_cnt_date dt_cnt_last_visit,buy_cnt_date dt_cnt_last_buy,buy_cnt_date ratio_spc,buy_cnt_date refund_rate,buy_cnt_date wmp_freq,buy_cnt_date wmp_stay_avg,buy_cnt_date wmp_pick_click,buy_cnt_date rgt_elapsed_days,buy_cnt_date buy_amount_avg_all,buy_cnt_date text_mail_wemakepay,buy_cnt_date click_cnt,buy_cnt_date search_cnt,buy_cnt_date download_coupon_cnt,buy_cnt_date cart_cnt,buy_cnt_date cart_buy_cnt,buy_cnt_date cart_category_cnt,buy_cnt_date uv_cnt,buy_cnt_payment^2,buy_cnt_payment buy_amount,buy_cnt_payment buying_trend,buy_cnt_payment cate_buy_cnt,buy_cnt_payment cate_rate,buy_cnt_payment coupon_use_rate,buy_cnt_payment buy_amount_3day,buy_cnt_payment buy_cnt_payment_3day,buy_cnt_payment buy_amount_avg,buy_cnt_payment dt_cnt_last_visit,buy_cnt_payment dt_cnt_last_buy,buy_cnt_payment ratio_spc,buy_cnt_payment refund_rate,buy_cnt_payment wmp_freq,buy_cnt_payment wmp_stay_avg,buy_cnt_payment wmp_pick_click,buy_cnt_payment rgt_elapsed_days,buy_cnt_payment buy_amount_avg_all,buy_cnt_payment text_mail_wemakepay,buy_cnt_payment click_cnt,buy_cnt_payment search_cnt,buy_cnt_payment download_coupon_cnt,buy_cnt_payment cart_cnt,buy_cnt_payment cart_buy_cnt,buy_cnt_payment cart_category_cnt,buy_cnt_payment uv_cnt,buy_amount^2,buy_amount buying_trend,buy_amount cate_buy_cnt,buy_amount cate_rate,buy_amount coupon_use_rate,buy_amount buy_amount_3day,buy_amount buy_cnt_payment_3day,buy_amount buy_amount_avg,buy_amount dt_cnt_last_visit,buy_amount dt_cnt_last_buy,buy_amount ratio_spc,buy_amount refund_rate,buy_amount wmp_freq,buy_amount wmp_stay_avg,buy_amount wmp_pick_click,buy_amount rgt_elapsed_days,buy_amount buy_amount_avg_all,buy_amount text_mail_wemakepay,buy_amount click_cnt,buy_amount search_cnt,buy_amount download_coupon_cnt,buy_amount cart_cnt,buy_amount cart_buy_cnt,buy_amount cart_category_cnt,buy_amount uv_cnt,buying_trend^2,buying_trend cate_buy_cnt,buying_trend cate_rate,buying_trend coupon_use_rate,buying_trend buy_amount_3day,buying_trend buy_cnt_payment_3day,buying_trend buy_amount_avg,buying_trend dt_cnt_last_visit,buying_trend dt_cnt_last_buy,buying_trend ratio_spc,buying_trend refund_rate,buying_trend wmp_freq,buying_trend wmp_stay_avg,buying_trend wmp_pick_click,buying_trend rgt_elapsed_days,buying_trend buy_amount_avg_all,buying_trend text_mail_wemakepay,buying_trend click_cnt,buying_trend search_cnt,buying_trend download_coupon_cnt,buying_trend cart_cnt,buying_trend cart_buy_cnt,buying_trend cart_category_cnt,buying_trend uv_cnt,cate_buy_cnt^2,cate_buy_cnt cate_rate,cate_buy_cnt coupon_use_rate,cate_buy_cnt buy_amount_3day,cate_buy_cnt buy_cnt_payment_3day

In [36]:
data_permut2_poly3.shape

(419554, 4497)

In [37]:
data_permut2_poly3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 419554 entries, 0 to 419553
Columns: 4497 entries, start_date to uv_cnt^3
dtypes: float64(4494), int64(2), object(1)
memory usage: 14.1+ GB


---

### - 날짜 index 활용하기

In [38]:
date_array = np.sort( tmp.start_date.unique() )

---
---

# 2. 통계량 살피기
- trend_corr & error_mse_
- trend_corr : 2020_0113 폴더의 poly3_random_stats.pkl 파일 이용

## 2.1 trend_corr
- 전체 데이터에서, 8:2 랜덤 sampling (50번 / 300번 / 300번_v2)

In [40]:
pwd

'/mnt/sda1/myeonggyulee/myeonggyulee/uplift/2020_0113/ver5_0203_trendcorr'

In [41]:
glob.glob('../poly3*.pkl')

['../poly3_date_stats.pkl', '../poly3_random_stats.pkl']

In [42]:
glob.glob('*.pkl')

['data_parallel_poly3_test_0211.pkl',
 'data_parallel_test_0210.pkl',
 'data_parallel_test_0207.pkl']

In [43]:
# 50번
import pickle

with open('../poly3_random_stats.pkl', 'rb') as f:
    trend_corr = pickle.load(f)

In [44]:
# 300번
import pickle

with open('data_parallel_test_0210.pkl', 'rb') as f:
    trend_corr_300 = pickle.load(f)

In [45]:
# 300번_v2
import pickle

with open('data_parallel_poly3_test_0211.pkl', 'rb') as f:
    trend_corr_300_v2 = pickle.load(f)

In [46]:
len(trend_corr)

50

In [47]:
trend_corr[0]

,Feature,Trend_changes,Trend_changes_test,Trend_correlation
0,result_buy_yn,0,0,0.000000
1,1,0,0,0.000000
2,buy_period,5,5,-0.284964
3,buy_cnt_date,6,7,0.345848
4,buy_cnt_payment,4,7,0.312874
...,...,...,...,...
4491,cart_buy_cnt uv_cnt^2,6,6,0.016934
4492,cart_category_cnt^3,5,5,-0.079416
4493,cart_category_cnt^2 uv_cnt,6,5,0.304310
4494,cart_category_cnt uv_cnt^2,5,4,-0.130260


In [48]:
trend_corr[1].head()

,Feature,Trend_changes,Trend_changes_test,Trend_correlation
0,result_buy_yn,0,0,0.000000
1,1,0,0,0.000000
2,buy_period,7,4,0.408708
3,buy_cnt_date,4,6,0.236592
4,buy_cnt_payment,6,5,0.337630


In [50]:
# df들을 concat 해서 활용하자
trend_corr_concat = pd.concat( trend_corr, axis=0 )

In [51]:
trend_corr_concat.shape

(224800, 4)

In [52]:
# feature 별로 mean값 구해야해

trend_corr_concat.groupby('Feature').mean()

,Trend_changes,Trend_changes_test,Trend_correlation
Feature,,,
1,0.00,0.00,0.000000
buy_amount,4.24,5.02,0.188993
buy_amount buy_amount_3day,5.66,5.22,-0.011307
buy_amount buy_amount_3day buy_amount_avg,5.34,5.18,-0.119168
buy_amount buy_amount_3day buy_amount_avg_all,4.56,5.14,0.014466
...,...,...,...
wmp_stay_avg^2 search_cnt,3.80,5.16,-0.239362
wmp_stay_avg^2 text_mail_wemakepay,4.72,5.26,0.019325
wmp_stay_avg^2 uv_cnt,5.42,5.12,-0.215012


In [53]:
trend_corr_concat_ = trend_corr_concat.groupby('Feature').mean()

In [54]:
trend_corr_concat_.sort_values(by='Trend_correlation', ascending=False).head(10)

,Trend_changes,Trend_changes_test,Trend_correlation
Feature,,,
buy_cnt_payment_3day^2,1.86,1.32,0.527085
buy_cnt_payment_3day,1.86,1.32,0.527085
buy_cnt_payment_3day^3,1.86,1.32,0.527085
dt_cnt_last_buy wmp_freq text_mail_wemakepay,3.50,4.26,0.516259
text_mail_wemakepay search_cnt,6.84,5.44,0.504132
dt_cnt_last_buy refund_rate search_cnt,5.00,4.82,0.494415
cate_rate refund_rate search_cnt,4.88,5.26,0.492850
dt_cnt_last_buy refund_rate click_cnt,5.60,5.26,0.492109
ratio_spc wmp_pick_click rgt_elapsed_days,5.84,5.14,0.489932


In [55]:
trend_corr_concat_.shape

(4496, 3)

----
---

### ### trend_corr_300 / trend_corr_300_v2
- trend_corr_300에는 (add, substract, division, poly3) 데이터가 모두 존재. But trend_corr_300_v2 에는 poly3의 데이터만 존재.
- 리스트 하나에 20개의 parallel 결과 저장
- 변수들의 nunique 통계량도 저장

#### - nunique 구하기

In [39]:
data_permuted = [data_permut2_add, data_permut2_subs, data_permut2_divis, data_permut2_poly3]

In [57]:
data_permuted[0].drop(columns = ['start_date', 'result_buy_yn', 'cont_group']).nunique().to_frame(name='nunique_')

,nunique_
buy_period + buy_cnt_date,400
buy_period + buy_cnt_payment,527
buy_period + buy_amount,370293
buy_period + buying_trend,7888
buy_period + cate_buy_cnt,483
...,...
cart_cnt + cart_category_cnt,325
cart_cnt + uv_cnt,413
cart_buy_cnt + cart_category_cnt,222
cart_buy_cnt + uv_cnt,291


In [60]:
nunique_stat = pd.DataFrame()
for data_ in data_permuted :
    nunique_stat = pd.concat( [nunique_stat, data_.drop(columns = ['start_date', 'result_buy_yn', 'cont_group']).nunique().to_frame(name='nunique_') ], axis=0)

In [61]:
nunique_stat.head()

,nunique_
buy_period + buy_cnt_date,400
buy_period + buy_cnt_payment,527
buy_period + buy_amount,370293
buy_period + buying_trend,7888
buy_period + cate_buy_cnt,483


In [62]:
nunique_stat.shape

(6384, 1)

---
#### - trend_corr mean 구하기

In [63]:
print(len( trend_corr_300 ), len(trend_corr_300_v2) )

300 300


In [12]:
len(trend_corr_300[0])

20

In [13]:
pd.concat( trend_corr_300[0], axis=0)

,Feature,Trend_changes,Trend_changes_test,Trend_correlation
0,result_buy_yn,0,0,0.000000
1,buy_period + buy_cnt_date,4,8,0.014175
2,buy_period + buy_cnt_payment,6,6,-0.351646
3,buy_period + buy_amount,6,4,0.034977
4,buy_period + buying_trend,7,6,-0.164275
...,...,...,...,...
300,cart_buy_cnt uv_cnt^2,4,5,-0.461504
301,cart_category_cnt^3,3,7,0.374824
302,cart_category_cnt^2 uv_cnt,6,2,0.582840
303,cart_category_cnt uv_cnt^2,7,6,0.002136


In [14]:
print( 6404 * 300 )

1921200


In [16]:
pd.concat( [pd.concat(sampling, axis=0) for sampling in trend_corr_300] , axis=0 ).shape

(1921200, 4)

In [64]:
# make stat
trend_corr_300_stat    = pd.concat( [pd.concat(sampling, axis=0) for sampling in trend_corr_300]    , axis=0 ).reset_index(drop=True)
trend_corr_300_stat_v2 = pd.concat( [pd.concat(sampling, axis=0) for sampling in trend_corr_300_v2] , axis=0 ).reset_index(drop=True)

In [66]:
trend_corr_300_stat

,Feature,Trend_changes,Trend_changes_test,Trend_correlation
0,result_buy_yn,0,0,0.000000
1,buy_period + buy_cnt_date,4,8,0.014175
2,buy_period + buy_cnt_payment,6,6,-0.351646
3,buy_period + buy_amount,6,4,0.034977
4,buy_period + buying_trend,7,6,-0.164275
...,...,...,...,...
1921195,cart_buy_cnt uv_cnt^2,4,2,0.152896
1921196,cart_category_cnt^3,7,4,-0.322099
1921197,cart_category_cnt^2 uv_cnt,5,3,-0.011496
1921198,cart_category_cnt uv_cnt^2,4,5,-0.179856


In [65]:
trend_corr_300_stat_v2

,Feature,Trend_changes,Trend_changes_test,Trend_correlation
0,result_buy_yn,0,0,0.000000
1,buy_period,7,4,0.214859
2,buy_cnt_date,7,6,0.299585
3,buy_cnt_payment,6,3,0.001738
4,buy_amount,6,4,0.450598
...,...,...,...,...
1354195,cart_buy_cnt uv_cnt^2,4,4,0.117658
1354196,cart_category_cnt^3,6,5,0.057994
1354197,cart_category_cnt^2 uv_cnt,5,6,0.085267
1354198,cart_category_cnt uv_cnt^2,5,4,-0.113681


In [67]:
# 합치기 : trend_corr_300 + trend_corr_300_stat_v2

trend_corr_300_stat = pd.concat( [trend_corr_300_stat, trend_corr_300_stat_v2], axis=0).reset_index(drop=True)

In [68]:
# poly3 는 600개씩
trend_corr_300_stat.groupby('Feature').size().sort_values(ascending=False)

Feature
result_buy_yn                               12000
wmp_stay_avg^3                                600
buy_period buy_amount buy_amount_avg_all      600
buy_cnt_payment_3day^2 wmp_freq               600
buy_cnt_payment_3day^2 wmp_pick_click         600
                                            ...  
refund_rate - wmp_pick_click                  300
refund_rate - wmp_freq                        300
refund_rate - uv_cnt                          300
refund_rate - text_mail_wemakepay             300
cate_rate - cart_cnt                          300
Length: 6385, dtype: int64

In [69]:
trend_corr_300_stat.groupby('Feature').mean().sort_values(by='Trend_correlation', ascending=False).head(20)

,Trend_changes,Trend_changes_test,Trend_correlation
Feature,,,
text_mail_wemakepay - refund_rate,3.803333,4.556667,0.613091
text_mail_wemakepay - wmp_pick_click,2.786667,3.500000,0.535740
ratio_spc / wmp_stay_avg,5.880000,5.346667,0.518841
rgt_elapsed_days cart_category_cnt uv_cnt,5.271667,5.176667,0.518836
buy_cnt_payment_3day dt_cnt_last_buy uv_cnt,3.713333,4.485000,0.476912
text_mail_wemakepay search_cnt,6.790000,5.468333,0.474682
buy_period cate_buy_cnt dt_cnt_last_buy,5.093333,5.216667,0.469810
click_cnt / wmp_stay_avg,4.940000,4.996667,0.467323
dt_cnt_last_buy refund_rate wmp_freq,4.921667,5.341667,0.460568


In [70]:
trend_corr_300_mean = trend_corr_300_stat.groupby('Feature').mean().sort_values(by='Trend_correlation', ascending=False)

In [71]:
# nunique 통계량이랑 같이 살피기
trend_corr_300_mean.merge(nunique_stat, left_index=True, right_index=True)

,Trend_changes,Trend_changes_test,Trend_correlation,nunique_
text_mail_wemakepay - refund_rate,3.803333,4.556667,0.613091,107
text_mail_wemakepay - wmp_pick_click,2.786667,3.500000,0.535740,75
ratio_spc / wmp_stay_avg,5.880000,5.346667,0.518841,571
rgt_elapsed_days cart_category_cnt uv_cnt,5.271667,5.176667,0.518836,150855
buy_cnt_payment_3day dt_cnt_last_buy uv_cnt,3.713333,4.485000,0.476912,6609
...,...,...,...,...
buying_trend^2,1.800000,1.961667,-0.733413,399
cate_rate buy_amount_3day download_coupon_cnt,5.453333,5.221667,-0.766748,106553
buy_cnt_date / buy_cnt_payment,4.896667,4.346667,-0.776823,3584
buy_cnt_date - buy_cnt_payment,4.613333,4.280000,-0.879652,212


In [72]:
trend_corr_300_mean_nunique = trend_corr_300_mean.merge(nunique_stat, left_index=True, right_index=True)

---

In [73]:
# trim_mean 통계량도 같이 저장해준다
trend_corr_300_stat.groupby('Feature')['Trend_correlation'].apply(lambda x: scipy.stats.trim_mean(x, proportiontocut = 0.05) )

Feature
buy_amount                            0.185220
buy_amount + buy_amount_3day          0.083051
buy_amount + buy_amount_avg           0.190206
buy_amount + buy_amount_avg_all       0.158298
buy_amount + buy_cnt_payment_3day     0.210987
                                        ...   
wmp_stay_avg^2 search_cnt            -0.196421
wmp_stay_avg^2 text_mail_wemakepay   -0.034580
wmp_stay_avg^2 uv_cnt                -0.289885
wmp_stay_avg^2 wmp_pick_click         0.085741
wmp_stay_avg^3                        0.184831
Name: Trend_correlation, Length: 6385, dtype: float64

In [74]:
trim_mean_of_trendcorr = trend_corr_300_stat.groupby('Feature')['Trend_correlation'].apply(lambda x: scipy.stats.trim_mean(x, proportiontocut = 0.1) ).to_frame(name = 'trim_mean_of_trendcorr')

In [75]:
trim_mean_of_trendcorr.head()

,trim_mean_of_trendcorr
Feature,
buy_amount,0.187922
buy_amount + buy_amount_3day,0.082750
buy_amount + buy_amount_avg,0.192479
buy_amount + buy_amount_avg_all,0.160232
buy_amount + buy_cnt_payment_3day,0.215862


In [76]:
trend_corr_300_mean_nunique.merge(trim_mean_of_trendcorr, left_index=True, right_index=True)

,Trend_changes,Trend_changes_test,Trend_correlation,nunique_,trim_mean_of_trendcorr
text_mail_wemakepay - refund_rate,3.803333,4.556667,0.613091,107,0.660457
text_mail_wemakepay - wmp_pick_click,2.786667,3.500000,0.535740,75,0.606588
ratio_spc / wmp_stay_avg,5.880000,5.346667,0.518841,571,0.543170
rgt_elapsed_days cart_category_cnt uv_cnt,5.271667,5.176667,0.518836,150855,0.528839
buy_cnt_payment_3day dt_cnt_last_buy uv_cnt,3.713333,4.485000,0.476912,6609,0.485189
...,...,...,...,...,...
buying_trend^2,1.800000,1.961667,-0.733413,399,-0.783620
cate_rate buy_amount_3day download_coupon_cnt,5.453333,5.221667,-0.766748,106553,-0.787203
buy_cnt_date / buy_cnt_payment,4.896667,4.346667,-0.776823,3584,-0.837213
buy_cnt_date - buy_cnt_payment,4.613333,4.280000,-0.879652,212,-0.922761


In [77]:
trend_corr_300_mean_nunique = trend_corr_300_mean_nunique.merge(trim_mean_of_trendcorr, left_index=True, right_index=True)

In [79]:
# trend_corr_300_mean_nunique_ver3
trend_corr_300_mean_nunique.to_csv('../trend_corr_300_mean_nunique.csv')